# Imports

In [1]:
import pandas as pd
import requests
import numpy as np
from mip import Model, xsum, minimize, BINARY, maximize
from pandas.tseries.offsets import BDay

# Preprocessing

## Calender data

In [2]:
def get_dates():
    today = pd.to_datetime('today')
    dates = pd.date_range(start=today+BDay(1), end=today + BDay(5))
    return dates
dates = get_dates()

In [3]:
def get_calender_data(dates):
    """Return number of appointments for the next 5 days"""
    df_cal = pd.DataFrame(index=dates, columns=['appointments'], data=[0,1,1,3,0])
    return df_cal
df_cal = get_calender_data(dates)

## Mobility data

In [4]:
def get_mobility_data(dates):
    """
    TODO: international possible default here Germany
    """
    url = f'https://covid19-static.cdn-apple.com/covid19-mobility-data/2019HotfixDev24/v3/en-us/applemobilitytrends-2020-10-29.csv'
    df_mobility = pd.read_csv(url)
    
    map_df = df_mobility[df_mobility['region'] == "Germany"].T[6:].rename(columns={45: "driving", 46: "transit", 47: "walking"})
    map_df.index = pd.to_datetime(map_df.index)
    map_df = map_df.loc[(map_df.index >= '2020-10-19') & (map_df.index <= '2020-10-23')]
    map_df.index = pd.to_datetime(dates)
    return map_df

df_mobility = get_mobility_data(dates)
df_mobility

/Users/timloehr/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,driving,transit,walking
2020-11-02 17:03:15.109273,115.13,120.19,124.19
2020-11-03 17:03:15.109273,115.65,119.52,121.63
2020-11-04 17:03:15.109273,113.77,118.85,118.91
2020-11-05 17:03:15.109273,114.87,119.14,122.17
2020-11-06 17:03:15.109273,121.38,117.44,118.78


# Weather

## OpenWeather

In [5]:
api_key = '30d9085988663142ce4cb478d09e6d00'
location = (49.4521, 11.0767) # Nürnberg
url = f'https://api.openweathermap.org/data/2.5/onecall?lat={location[0]}&lon={location[1]}&exclude=current,minutely,hourly&appid={api_key}&units=metric'

In [6]:
def forecast(lat, lon, dates):
    location = (lat, lon)
    url = f'https://api.openweathermap.org/data/2.5/onecall?lat={location[0]}&lon={location[1]}&exclude=current,minutely,hourly&appid={api_key}&units=metric'
    r = requests.get(url)
    
    # prepare df
    columns = ['date', 'temp_min', 'temp_max', 'temp_morning', 'temp_day',\
           'temp_evening', 'pressure', 'humidity', 'dew_point',\
           'wind_speed', 'pop', 'uvi']
    df_forecast = pd.DataFrame(columns=[columns])
    
    # loop through days
    for i, day in enumerate(r.json()['daily']):
        df_forecast.loc[i] = [day['dt'], day['temp']['min'], day['temp']['max'], \
                               day['temp']['morn'], day['temp']['day'], day['temp']['eve'], \
                               day['pressure'], day['humidity'], day['dew_point'], \
                               day['wind_speed'], day['pop'], day['uvi']]
    
    # data cleaning
    df_forecast.columns = columns
    df_forecast['date'] = df_forecast['date'].astype(str).str[:-2]
    df_forecast['date'] = pd.to_datetime(df_forecast['date'], unit='s')
    df_forecast['weekday'] = df_forecast['date'].dt.weekday < 5

    df_forecast = df_forecast.loc[df_forecast['weekday']==True]
    df_forecast.index = pd.to_datetime(dates)
    df_forecast = df_forecast.drop('date', axis=1)
    
    return df_forecast

In [7]:
df_forecast = forecast(49.4521, 11.0767, dates)

In [8]:
df_forecast

,temp_min,temp_max,temp_morning,temp_day,temp_evening,pressure,humidity,dew_point,wind_speed,pop,uvi,weekday
2020-11-02 17:03:15.109273,12.81,18.04,12.81,15.96,16.23,1017.0,86.0,13.71,3.49,0.28,1.48,True
2020-11-03 17:03:15.109273,7.20,15.05,13.55,10.75,10.38,1027.0,69.0,5.42,3.73,0.31,1.42,True
2020-11-04 17:03:15.109273,3.58,9.75,4.23,7.46,8.50,1031.0,72.0,2.79,0.80,0.00,1.27,True
2020-11-05 17:03:15.109273,3.99,10.71,6.28,7.52,9.05,1037.0,65.0,1.40,2.62,0.00,1.23,True
2020-11-06 17:03:15.109273,2.16,10.24,2.55,6.74,8.30,1036.0,63.0,0.34,4.01,0.00,1.21,True


# Merge data

In [9]:
df_merged = pd.concat([df_cal, df_mobility, df_forecast], axis=1)
colums = ["appointments","driving",	"transit", "temp_day",	"humidity", "wind_speed", "pop", "uvi"]
df_merged = df_merged[colums]

In [10]:
def merge_data_create_model(df):
    df = df.copy()
    for col in df.columns:
        # if col=='appointments':
        #    continue

        
        if col in ['driving', 'transit', 'wind_speed', 'pop']:
            # if values are high I like to stay at home thus inverting
            df[col] = -1*df[col] 
        elif col=='temp_day' or col=='humidity':
            # stay at home if differ from mean
            df[col] = -1*(df[col] - df[col].mean()).abs()
        
        df[col]=(df[col]-df[col].min())/(df[col].max()-df[col].min())

    y = df.sum(axis=1).to_numpy()
    return y, df


## Optimization starting here
Using the "knapsack" model from Mixed Integer linear programming for solving this.

Model: $y_d = N_a + (23 - T) + p + m$

where $N_a$ number of appointment, $T$ temperature, $p$ precipitation, $m$ mobility.

We want to minimize $\sum y$ under the constrain, that we will go to office more than 3 times a week

Minimize: $\sum_d y_d \cdot w_d$

Subject to: $ \sum_d w_d >= 3$ with $w_d \in \{0,1\} \ \forall d\in I$

The output weights $w_d$ are 1 (should go to office) or 0 (better stay at home)

In [11]:
def mip_optimization(y, constrain=3, daily_weights=None):
    """Mixed integer linear programming optimization with constraints.
    Args:
        y (numpy.ndarray): sum of daily features (dim=#ofdays)
        constrain (int): minimum days in office
        daily_weights (array): weighting of days, e.g. if you prefer to come on mondays
    Return:
         
    """
    # daily weighting
    u = np.ones(len(y)) if daily_weights==None else daily_weights
    I = range(len(y))   # idx for days for summation

    m = Model("knapsack")   # MIP model
    w = [m.add_var(var_type=BINARY) for i in I] # weights to optimize
    m.objective  = maximize(xsum(y[i]* w[i] for i in I)) # optimization function
    m += xsum(w[i] * u[i] for i in I) <= constrain # constraint 
    m.optimize()

    #selected = [i for i in I if w[i].x >= 0.99]
    selected = [w[i].x for i in I]
    return selected

In [12]:
# run model and optimization
y, df_week = merge_data_create_model(df_merged)
selected = mip_optimization(y, constrain=2)
print(selected)
print(df_week.sum(axis=1))
df_merged


[0.0, 0.0, 1.0, 1.0, 0.0]
2020-11-02 17:03:15.109273    2.080056
2020-11-03 17:03:15.109273    4.123503
2020-11-04 17:03:15.109273    5.819796
2020-11-05 17:03:15.109273    5.175708
2020-11-06 17:03:15.109273    3.138772
Freq: D, dtype: float64


,appointments,driving,transit,temp_day,humidity,wind_speed,pop,uvi
2020-11-02 17:03:15.109273,0,115.13,120.19,15.96,86.0,3.49,0.28,1.48
2020-11-03 17:03:15.109273,1,115.65,119.52,10.75,69.0,3.73,0.31,1.42
2020-11-04 17:03:15.109273,1,113.77,118.85,7.46,72.0,0.80,0.00,1.27
2020-11-05 17:03:15.109273,3,114.87,119.14,7.52,65.0,2.62,0.00,1.23
2020-11-06 17:03:15.109273,0,121.38,117.44,6.74,63.0,4.01,0.00,1.21


In [13]:
df_week

,appointments,driving,transit,temp_day,humidity,wind_speed,pop,uvi
2020-11-02 17:03:15.109273,0.000000,0.821288,0,0.000000,0.000000,0.161994,0.096774,1.000000
2020-11-03 17:03:15.109273,0.333333,0.752957,0.243636,1.000000,0.928571,0.087227,0.000000,0.777778
2020-11-04 17:03:15.109273,0.333333,1,0.487273,0.776967,1.000000,1.000000,1.000000,0.222222
2020-11-05 17:03:15.109273,1.000000,0.855453,0.381818,0.788484,0.642857,0.433022,1.000000,0.074074
2020-11-06 17:03:15.109273,0.000000,0,1,0.638772,0.500000,0.000000,1.000000,0.000000
